In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
from PIL import Image
import glob
import os
import time
import tensorflow as tf
import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
import matplotlib.pyplot as plt
from keras.models import load_model
from keras.preprocessing import image
import json
from skimage import io

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# CHANGE THE VALUE OF "folder_path" TO YOUR CORRECT DIRECTORY
folder_path = "/content/drive/MyDrive/Documentation Files/(4)Step-by-step/prediction_files/"

# NO NEED TO CHANGE THESE VALUES
models_path = folder_path + "models/"
input_image_path = folder_path + "input_image/"
roi_path = folder_path + "roi/"
masked_digits_path = folder_path + "masked_digits/"
cropped_digits_path = folder_path + "cropped_digits/"
model_reading = models.resnext50_32x4d(pretrained=True)    
num_ftrs = model_reading.fc.in_features
model_reading.fc = nn.Linear(num_ftrs, 10)
checkpoint = torch.load(models_path + "digit_reading_model.pt")
model_reading.load_state_dict(checkpoint)

In [ ]:
indexes = [0,1,2,3,4,5,6,7,8,9]
for range in indexes:
  for filename in os.listdir("/content/drive/MyDrive/Digit Reading Files/Digits Dataset for F1/" + str(range)):
    image = cv2.imread("/content/drive/MyDrive/Digit Reading Files/Digits Dataset for F1/" + str(range) + "/" + filename)
    cv2_imshow(image)
    model_reading.eval()
    with torch.no_grad():
      prediction = predict_image("/content/drive/MyDrive/Digit Reading Files/Digits Dataset for F1/" + str(range) + "/" + filename)
      print("Predicted Class: ",prediction)
    matrix[int(prediction)][range] += 1
    print(matrix)
    #print(filename)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
matrix_copy = matrix

In [ ]:
print("CONFUSION MATRIX")
print(matrix)

CONFUSION MATRIX
[[2439    2    1    1    3    2    4    0    0    2]
 [   2  542    4    1    5    0    0    5    0    1]
 [   2    0  472    2    1    0    1    0    0    3]
 [   1    1    1  496    1    4    0    0    0    2]
 [   0    2    3    0  392    1    2    0    0    0]
 [   1    0    1    4    0  354    2    0    2    0]
 [   4    0    2    1    1    1  390    0    0    2]
 [   0   10    5    1    5    0    0  366    0    0]
 [   1    0    1    2    1    5    6    0  384    5]
 [   4    0    0    3    0    3    0    1    6  328]]


In [ ]:
print("CONFUSION MATRIX")
print(matrix)
print("\n")

# GET THE PRECISION AND RECALL OF EACH CLASS
precision_list = [0,0,0,0,0,0,0,0,0,0]
recall_list = [0,0,0,0,0,0,0,0,0,0]
indexes = [0,1,2,3,4,5,6,7,8,9]

for index1 in indexes:
  numerator = 0
  denominator_row = 0
  denominator_column = 0
  for index2 in indexes:
    numerator = matrix[index1][index1]
    denominator_row += matrix[index1][index2]
    denominator_column += matrix[index2][index1]
  precision_list[index1] = numerator/denominator_row
  recall_list[index1] = numerator/denominator_column

indexes = [0,1,2,3,4,5,6,7,8,9]
print("PRECISION AND RECALL TABLE OF EACH CLASS")
for index in indexes:
  print(str(index) + "   " + str(precision_list[index]) + "   " +str(recall_list[index]))
print("\n")
# GET THE F1 OF EACH CLASS
f1_list = [0,0,0,0,0,0,0,0,0,0]
print("F1 SCORE OF EACH CLASS")
for index in indexes:
  f1_list[index] = 2*((precision_list[index]*recall_list[index])/(precision_list[index]+recall_list[index]))
  print(str(index) + "   " + str(f1_list[index]))
print("\n")
# GET THE MACRO F1
f1_macro = sum(f1_list)/10
print("MACRO F1 SCORE")
print(f1_macro)  
print("\n")
# GET LEN OF EACH CLASS AND THE TOTAL LEN
indexes = [0,1,2,3,4,5,6,7,8,9]
len_list = [0,0,0,0,0,0,0,0,0,0]
total_len = 0
for index in indexes:
  len = 0
  for filename in os.listdir("/content/drive/MyDrive/Digit Reading Files/Digits Dataset for F1/" + str(index)):
    len +=1
    total_len +=1
  len_list[index] = len


# GET THE WEIGHTED F1
print("WEIGHTED F1 SCORE")
indexes = [0,1,2,3,4,5,6,7,8,9]
f1_weighted_numerator = 0
for index in indexes:
  weight = len_list[index]*f1_list[index]
  f1_weighted_numerator += weight
f1_weighted = f1_weighted_numerator/total_len
print(f1_weighted)  

CONFUSION MATRIX
[[2439    2    1    1    3    2    4    0    0    2]
 [   2  542    4    1    5    0    0    5    0    1]
 [   2    0  472    2    1    0    1    0    0    3]
 [   1    1    1  496    1    4    0    0    0    2]
 [   0    2    3    0  392    1    2    0    0    0]
 [   1    0    1    4    0  354    2    0    2    0]
 [   4    0    2    1    1    1  390    0    0    2]
 [   0   10    5    1    5    0    0  366    0    0]
 [   1    0    1    2    1    5    6    0  384    5]
 [   4    0    0    3    0    3    0    1    6  328]]


PRECISION AND RECALL TABLE OF EACH CLASS
0   0.9938875305623472   0.9938875305623472
1   0.9678571428571429   0.9730700179533214
2   0.9812889812889813   0.963265306122449
3   0.9802371541501976   0.9706457925636007
4   0.98   0.9584352078239609
5   0.9725274725274725   0.9567567567567568
6   0.972568578553616   0.9629629629629629
7   0.9457364341085271   0.9838709677419355
8   0.9481481481481482   0.9795918367346939
9   0.9507246376811594   0.95

In [ ]:
# GET THE F1 OF EACH CLASS
f1_list = [0,0,0,0,0,0,0,0,0,0]
print("F1 SCORE OF EACH CLASS")
for index in indexes:
  f1_list[index] = 2*((precision_list[index]*recall_list[index])/(precision_list[index]+recall_list[index]))
  print(str(index) + "   " + str(f1_list[index]))

F1 SCORE OF EACH CLASS
0   0.9938875305623472
1   0.9704565801253356
2   0.9721936148300722
3   0.975417895771878
4   0.969097651421508
5   0.9645776566757492
6   0.9677419354838711
7   0.9644268774703558
8   0.9636135508155584
9   0.9534883720930233


In [ ]:
# GET THE MACRO F1
f1_macro = sum(f1_list)/10
print("MACRO F1 SCORE")
print(f1_macro)

MACRO F1 SCORE
0.9694901665249699


In [ ]:
# GET LEN OF EACH CLASS AND THE TOTAL LEN
indexes = [0,1,2,3,4,5,6,7,8,9]
len_list = [0,0,0,0,0,0,0,0,0,0]
total_len = 0
for index in indexes:
  len = 0
  for filename in os.listdir("/content/drive/MyDrive/Digit Reading Files/Digits Dataset for F1/" + str(index)):
    len +=1
    total_len +=1
  len_list[index] = len
print(len_list)
print(total_len)

[2454, 557, 490, 511, 409, 370, 405, 372, 392, 343]
6303


In [ ]:
# GET THE WEIGHTED F1
print("WEIGHTED F1 SCORE")
indexes = [0,1,2,3,4,5,6,7,8,9]
f1_weighted_numerator = 0
for index in indexes:
  weight = len_list[index]*f1_list[index]
  f1_weighted_numerator += weight
f1_weighted = f1_weighted_numerator/total_len
print(f1_weighted)

WEIGHTED F1 SCORE
0.9778038567943753


In [ ]:
list_predicted = [0,0,0,0,0,0,0,0,0,0]

In [ ]:
import numpy
matrix = numpy.array([list_predicted, list_predicted,list_predicted,list_predicted,list_predicted,list_predicted,list_predicted,
                      list_predicted,list_predicted,list_predicted])
print(matrix)

In [ ]:
def predict_image(image_path): # PREDICTS THE IMAGE USING digit_reading_model.pt
  transformation = transforms.Compose([
      transforms.ToPILImage(),                                 
      transforms.Resize((32,32)),
      #transforms.RandomHorizontalFlip(),
      transforms.ToTensor(),
      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
      ])
  image_tensor = transformation(image).float()
  image_tensor = image_tensor.unsqueeze_(0)

  if torch.cuda.is_available():
      image_tensor.cuda()

  #input = Variable(image_tensor)
  output = model_reading(image_tensor)
  #print(output)
  index = output.data.numpy().argmax()
  return index